In [1]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

In [4]:
path = "data/parts/"

# Load data using pandas
cpus = pd.read_csv(path + "CPU_Data.csv")
gpus = pd.read_csv(path + "GPU_Data.csv")
rams = pd.read_csv(path + "RAM_Data.csv")

motherboards = pd.read_csv(path + "MOBO_Data.csv")
psus = pd.read_csv(path + "PSU_Data.csv")
cases = pd.read_csv(path + "Case_Data.csv")
storage = pd.read_csv(path + "Storage_Data.csv")

games = pd.read_csv("data/games/top300.csv")

# Return original value if conversion fails
def safe_to_numeric(val):
    try:
        return pd.to_numeric(val)
    except (ValueError, TypeError):
        return val  

# Convert numeric columns from string to numeric types
cpus = cpus.apply(safe_to_numeric)
gpus = gpus.apply(safe_to_numeric)
rams = rams.apply(safe_to_numeric)
cases = cases.apply(safe_to_numeric)
motherboards = motherboards.apply(safe_to_numeric)
psus = psus.apply(safe_to_numeric)
storage = storage.apply(safe_to_numeric)
games = games.apply(safe_to_numeric)

In [ ]:
display(cpus)
display(cpus)

,Brand,Name,Chipset,Score,Price,URL,Timestamp,Note
0,AMD,Athlon 3000G,AM4,4452,219,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07,NaN
1,Intel,Core i3-12100F,LGA1700,14090,339,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:09,NaN
2,Intel,Core i3-14100,FCLGA1700,15378,469,https://czone.my/czone/bx8071514100-1739103180...,2025-03-19 14:21:03,NaN
3,Intel,Core i3-14100,LGA1700,15082,519,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:09,NaN
4,Intel,Core i3-14100F,LGA1700,15378,379,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:09,NaN
...,...,...,...,...,...,...,...,...
110,AMD,Ryzen 9 7950X3D,AM5,62453,2999,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:09,NaN
111,AMD,Ryzen 9 9900X,AM5,54804,2149,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:09,NaN
112,AMD,Ryzen 9 9900X,AM5,54804,1999,https://czone.my/czone/100-100000662wof-173910...,2025-03-19 14:21:03,NaN
113,AMD,Ryzen 9 9950X,AM5,66310,3049,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:09,NaN
